In [1]:
import pandas as pd

/home/marifra/miniconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
import packaging
from openai import OpenAI
client = OpenAI(api_key="OPENAI_API_KEY")

In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_train.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES


In [4]:
categories = df_train['category'].drop_duplicates().to_list()

In [ ]:
len(categories)

1588

In [18]:
df_train['category'].value_counts()

category
PANTS                                   35973
COFFEE_MAKERS                           35104
BABY_CAR_SEATS                          34163
MUSICAL_KEYBOARDS                       33222
MATTRESSES                              32967
                                        ...  
CONSTRUCTION_LIME_BAGS                    206
COLD_FOOD_AND_DRINK_VENDING_MACHINES      162
PAINTBALL_SMOKE_GRENADES                  154
COMMERCIAL_POPCORN_MACHINES               141
HAMBURGER_FORMERS                         109
Name: count, Length: 1588, dtype: int64

##### Create examples of the top 10 categoties

In [41]:
top_categories = df_train['category'].value_counts().head(10).reset_index()

In [42]:
examples = df_train[['title', 'category']].drop_duplicates('category', keep='first')

In [46]:
examples_filtered = examples.merge(top_categories[['category']], on= 'category', how='inner')
examples_filtered

,title,category
0,Teclado Casio Wk-240 76 Teclas Profissional St...,MUSICAL_KEYBOARDS
1,Heladera Gafa 380 Impecable Urgente,REFRIGERATORS
2,Butaca 6 Cuotas Sin Interes!! Para Auto Bebes...,BABY_CAR_SEATS
3,Rottwalier Femea,PUREBRED_DOGS
4,Cafetera De Filtro Hamilton Beach 12 Tazas,COFFEE_MAKERS
5,Colchon 2 Plazas Usado Muy Buen Estado,MATTRESSES
6,Calça Feminina Pra Mulher Gravidas Dois Bolso...,PANTS
7,Campera De Cuero Moto Prosnik,MOTORCYCLE_JACKETS
8,Mini Depilador Eletrico Cabelo Nariz Orelha Ba...,HAIR_CLIPPERS
9,Cocina A Gas Longvie 18601bf 60cm Blanca Lh Cu...,RANGES


In [ ]:
examples_filtered_json = examples_filtered.to_json(orient= "records")
examples_filtered_json

'[{"title":"Teclado Casio Wk-240 76 Teclas Profissional Standard","category":"MUSICAL_KEYBOARDS"},{"title":"Heladera Gafa 380 Impecable Urgente","category":"REFRIGERATORS"},{"title":"Butaca  6 Cuotas Sin Interes!! Para Auto Bebesit Hasta 25kg","category":"BABY_CAR_SEATS"},{"title":"Rottwalier Femea ","category":"PUREBRED_DOGS"},{"title":"Cafetera De Filtro Hamilton Beach 12 Tazas","category":"COFFEE_MAKERS"},{"title":"Colchon 2 Plazas Usado Muy Buen Estado","category":"MATTRESSES"},{"title":" Cal\\u00e7a Feminina Pra Mulher Gravidas Dois Bolso Cintura Alta ","category":"PANTS"},{"title":"Campera De Cuero Moto Prosnik","category":"MOTORCYCLE_JACKETS"},{"title":"Mini Depilador Eletrico Cabelo Nariz Orelha Barba - Com Led","category":"HAIR_CLIPPERS"},{"title":"Cocina A Gas Longvie 18601bf 60cm Blanca Lh Cuotas","category":"RANGES"}]'

In [59]:
few_examples = ""
for row_number, row in examples_filtered.iterrows():
    few_examples += f"Example {row_number+1} '{row['title']}': {row['category']}\n"

In [60]:
print(few_examples)

Example 1 'Teclado Casio Wk-240 76 Teclas Profissional Standard': MUSICAL_KEYBOARDS
Example 2 'Heladera Gafa 380 Impecable Urgente': REFRIGERATORS
Example 3 'Butaca  6 Cuotas Sin Interes!! Para Auto Bebesit Hasta 25kg': BABY_CAR_SEATS
Example 4 'Rottwalier Femea ': PUREBRED_DOGS
Example 5 'Cafetera De Filtro Hamilton Beach 12 Tazas': COFFEE_MAKERS
Example 6 'Colchon 2 Plazas Usado Muy Buen Estado': MATTRESSES
Example 7 ' Calça Feminina Pra Mulher Gravidas Dois Bolso Cintura Alta ': PANTS
Example 8 'Campera De Cuero Moto Prosnik': MOTORCYCLE_JACKETS
Example 9 'Mini Depilador Eletrico Cabelo Nariz Orelha Barba - Com Led': HAIR_CLIPPERS
Example 10 'Cocina A Gas Longvie 18601bf 60cm Blanca Lh Cuotas': RANGES



##### Create a function with the prompt to automate the classification request to AI

- System message: high-level, global instructions about the assistant’s role, style, and constraints.

- User message: the actual classification request, including the text to classify and categories.

In [61]:
def create_classification_prompt(text_to_classify, categories):
    category_list_str = ",".join(categories)
    prompt = (
        f"Here are some examples of classifications:\n"
        f"{few_examples}\n"
        f"Now classify the following MercadoLibre product title into one of these categories: {category_list_str}.\n"
        f"Text: \"{text_to_classify}\"\n"
        f"Classification:"
    )
    return prompt

In [64]:
user_inputs = [create_classification_prompt(title, categories) for title in df_train['title'].head()]

In [65]:
user_inputs

['Here are some examples of classifications:\nExample 1 \'Teclado Casio Wk-240 76 Teclas Profissional Standard\': MUSICAL_KEYBOARDS\nExample 2 \'Heladera Gafa 380 Impecable Urgente\': REFRIGERATORS\nExample 3 \'Butaca  6 Cuotas Sin Interes!! Para Auto Bebesit Hasta 25kg\': BABY_CAR_SEATS\nExample 4 \'Rottwalier Femea \': PUREBRED_DOGS\nExample 5 \'Cafetera De Filtro Hamilton Beach 12 Tazas\': COFFEE_MAKERS\nExample 6 \'Colchon 2 Plazas Usado Muy Buen Estado\': MATTRESSES\nExample 7 \' Calça Feminina Pra Mulher Gravidas Dois Bolso Cintura Alta \': PANTS\nExample 8 \'Campera De Cuero Moto Prosnik\': MOTORCYCLE_JACKETS\nExample 9 \'Mini Depilador Eletrico Cabelo Nariz Orelha Barba - Com Led\': HAIR_CLIPPERS\nExample 10 \'Cocina A Gas Longvie 18601bf 60cm Blanca Lh Cuotas\': RANGES\n\nNow classify the following MercadoLibre product title into one of these categories: ELECTRIC_PRESSURE_WASHERS,SOUND_CARDS,WASHING_MACHINES,VEHICLE_BRAKE_DISCS,FALSE_EYELASHES,IRRIGATION_SPRINKLERS,RACQUETS,CA

##### Create a function to build the message in the expected format by the OpenAI Chat API and another to call the model and extract the responses

The message has two roles: system and user

In [62]:
def format_message(role, content):
    return {"role": role, "content": content}


def get_response(messages):
    completion = client.chat.completions.create(
        model="gpt-5-nano", #-- 'gpt-3.5-turbo',
        messages=messages,
    )
    content = completion.choices[0].message.content
    return content

In [63]:
system_message = (
"You are a product classifier for MercadoLibre product titles. "
"Titles may be in Spanish or Portuguese. "
"Always classify them into one of the provided categories, which are in English. "
"Return only the category name, without explanations or extra text. "
"If no category seems perfect, choose the closest match from the list. "
"Do not invent new categories."
)

##### Set messages using the system_message and the user_inputs as output of create_classification_prompt() function.

In [68]:
responses = []
for user_input in user_inputs:
    messages = [format_message("system", system_message),
                format_message("user", user_input)]
    response = get_response(messages)
    responses.append(response)

In [68]:
responses

['ELECTRIC_PRESSURE_WASHERS',
 'AUDIO_INTERFACES',
 'WASHING_MACHINES',
 'VEHICLE_BRAKE_DISCS',
 'FALSE_EYELASHES']

In [69]:
df_train.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES


? Fine Tuning GPT: This process adapts a pre-trained GPT model to a specific task by training it on your labeled data.